### Notebook to iterate through Logisitic Regression Model

In [196]:
# add justification for why Log Reg Model would be useful in this case

In [197]:
# import libraries
import pandas as pd
pd.options.display.max_columns = 50
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
# add import for Decision Treees
from sklearn.feature_selection import RFECV, SelectKBest, f_regression
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
import pickle

In [198]:
# loading in cleaned dataset
model_df = pd.read_csv('model_data.csv', index_col = 0)

In [199]:
model_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,Returning_Visitor,True,False


In [200]:
# Creating Dummys Columns from Cat
model_df = pd.get_dummies(model_df, columns=['SpecialDay', 'Month', 'OperatingSystems','Browser','Region','TrafficType','VisitorType','Weekend'])

In [201]:
model_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Revenue,SpecialDay_0.0,SpecialDay_0.2,SpecialDay_0.4,SpecialDay_0.6,SpecialDay_0.8,SpecialDay_1.0,Month_2,Month_3,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,...,TrafficType_1,TrafficType_2,TrafficType_3,TrafficType_4,TrafficType_5,TrafficType_6,TrafficType_7,TrafficType_8,TrafficType_9,TrafficType_10,TrafficType_11,TrafficType_12,TrafficType_13,TrafficType_14,TrafficType_15,TrafficType_16,TrafficType_17,TrafficType_18,TrafficType_19,TrafficType_20,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,False,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [202]:
# Dummying 'Revenue'
model_df['Revenue'] = model_df['Revenue'].astype(int)

In [203]:
model_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Revenue,SpecialDay_0.0,SpecialDay_0.2,SpecialDay_0.4,SpecialDay_0.6,SpecialDay_0.8,SpecialDay_1.0,Month_2,Month_3,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,...,TrafficType_1,TrafficType_2,TrafficType_3,TrafficType_4,TrafficType_5,TrafficType_6,TrafficType_7,TrafficType_8,TrafficType_9,TrafficType_10,TrafficType_11,TrafficType_12,TrafficType_13,TrafficType_14,TrafficType_15,TrafficType_16,TrafficType_17,TrafficType_18,TrafficType_19,TrafficType_20,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


## Train-Test Split

In [204]:
# Split data to be used in the models
X = model_df.drop('Revenue', axis = 1)

# Create target variable
y = model_df['Revenue'] # y is the column we're trying to predict

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)

In [206]:
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

## Baseline Logistic Regression Model

In [207]:
baseline_log_reg = LogisticRegression(solver='liblinear',max_iter=500, verbose=1, n_jobs=-1)

baseline_log_reg.fit(X_train, y_train)

log_reg_pred_train = baseline_log_reg.predict(X_train)
log_reg_pred_test = baseline_log_reg.predict(X_test)

print('Baseline Logistic Regression Model')

print("Training F1 Score: ", metrics.f1_score(y_train, log_reg_pred_train))
print("Testing F1 Score: ", metrics.f1_score(y_test, log_reg_pred_test))

print("Training Recall: ", metrics.recall_score(y_train, log_reg_pred_train))
print("Testing Recall: ", metrics.recall_score(y_test, log_reg_pred_test))

print("Training Accuracy Score:",metrics.accuracy_score(y_train, log_reg_pred_train))
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, log_reg_pred_test))

print("Training ROC, AUC Score:",metrics.roc_auc_score(y_train, log_reg_pred_train))
print("Testing ROC, AUC Score:",metrics.roc_auc_score(y_test, log_reg_pred_test))

print("Training Precision Score:",metrics.precision_score(y_train, log_reg_pred_train))
print("Testing Precision Score:",metrics.precision_score(y_test, log_reg_pred_test))

/Users/jefelito/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


[LibLinear]Baseline Logistic Regression Model
Training F1 Score:  0.5155020823692735
Testing F1 Score:  0.4826629680998614
Training Recall:  0.39363957597173144
Testing Recall:  0.35294117647058826
Training Accuracy Score: 0.8867740888936952
Testing Accuracy Score: 0.8790139474537788
Training ROC, AUC Score: 0.6847539044312181
Testing ROC, AUC Score: 0.6660458778105838
Training Precision Score: 0.7466487935656837
Testing Precision Score: 0.7631578947368421


## Logistic Regression Model with 'Balanced' Class Weight

In [214]:
balanced_log = LogisticRegression(class_weight='balanced', solver='lbfgs', verbose=1, n_jobs=-1)

balanced_log.fit(X_train, y_train)

balanced_log_pred_train = baseline_log_reg.predict(X_train)
balanced_log_pred_test = balanced_log.predict(X_test)

print('Logistic Regression Model with Balanced Class Weight')

print("Training F1 Score:",metrics.f1_score(y_train, balanced_log_pred_train))
print("Testing F1 Score:",metrics.f1_score(y_test, balanced_log_pred_test))

print("Training Accuracy Score:",metrics.accuracy_score(y_train, balanced_log_pred_train))
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, balanced_log_pred_test))

print("Training ROC,AUC Score:",metrics.roc_auc_score(y_train, balanced_log_pred_train))
print("Testing ROC,AUC Score:",metrics.roc_auc_score(y_test, balanced_log_pred_test))

print("Training Precision Score:",metrics.precision_score(y_train, balanced_log_pred_train))
print("Testing Precision Score:",metrics.precision_score(y_test, balanced_log_pred_test))

print("Training Precision Score:",metrics.precision_score(y_train, balanced_log_pred_train))
print("Testing Precision Score:",metrics.precision_score(y_test, balanced_log_pred_test))


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Logistic Regression Model with Balanced Class Weight
Training F1 Score: 0.5155020823692735
Testing F1 Score: 0.6021328958162427
Training Accuracy Score: 0.8867740888936952
Testing Accuracy Score: 0.842685695750892
Training ROC,AUC Score: 0.6847539044312181
Testing ROC,AUC Score: 0.8029059340418367
Training Precision Score: 0.7466487935656837
Testing Precision Score: 0.5055096418732782
Training Precision Score: 0.7466487935656837
Testing Precision Score: 0.5055096418732782


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.9s finished
